In [5]:
import mlflow
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("deploy-model-via-flask")

2022/07/24 14:23:47 INFO mlflow.tracking.fluent: Experiment with name 'deploy-model-via-flask' does not exist. Creating a new experiment.


<Experiment: artifact_location='./mlruns/2', experiment_id='2', lifecycle_stage='active', name='deploy-model-via-flask', tags={}>

In [17]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from xgboost import XGBClassifier
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
import pickle


### Load and prepare dataframe

In [7]:
from sklearn import datasets
iris = datasets.load_iris()
iris_df = pd.DataFrame(data=iris.data,columns=iris.feature_names)
iris_df.head(3)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2


In [8]:
iris.target_names

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [9]:
iris.target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [10]:
iris_df['target']  = iris.target
iris_df.target.value_counts()

0    50
1    50
2    50
Name: target, dtype: int64

In [11]:
iris_df.describe()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
count,150.000000,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333,1.000000
std,0.828066,0.435866,1.765298,0.762238,0.819232
min,4.300000,2.000000,1.000000,0.100000,0.000000
25%,5.100000,2.800000,1.600000,0.300000,0.000000
50%,5.800000,3.000000,4.350000,1.300000,1.000000
75%,6.400000,3.300000,5.100000,1.800000,2.000000
max,7.900000,4.400000,6.900000,2.500000,2.000000


## Segregate train and test dataset

In [12]:
x = iris_df.drop('target',axis=1)
y = iris_df['target']
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.33,random_state=42)

In [16]:
with mlflow.start_run():
    from sklearn import metrics
    mlflow.log_params({"developer" : "Arshad","ID":"XZZ7200","Dataset":"IRIS","random_state":"42","test_size":"0.33","algorithm":"XGBOOST"})
    xgb_classifer = XGBClassifier(objective='binary:logistic')
    xgb_classifer.fit(X_train,y_train)
    y_pred = xgb_classifer.predict(X_test)
    accuracy = metrics.accuracy_score(y_test,y_pred)
    metrics = {"Accuracy":accuracy}
    mlflow.log_metrics(metrics)

%%bash
$GIT_PYTHON_GIT_EXECUTABLE

In [22]:
pickle.dump(xgb_classifer,open('model','wb'))

In [27]:
model = pickle.load(open('model','rb'))

In [28]:
model.predict(X_test)

array([1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2,
       0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 1, 0, 0, 2, 1, 0, 0, 0, 2, 1, 1, 0,
       0, 1, 1, 2, 1, 2])

## Serve the model via Flask web app

In [18]:
%%writefile '/home/ec2-user/notebooks/scripts/predict.py'
from flask import Flask, request, jsonify
import pickle
import pandas as pd
import numpy as np
model = pickle.load(open('/home/ec2-user/notebooks/models/model','rb'))

def predictc(features):
    print("Inside predictc")
    predictions = model.predict(features)
    print(predictions)
    return predictions[0]
app = Flask('app')

@app.route('/predict',methods=['POST'])
def predict_endpoint():
    features = request.get_json()
    print("Inside predict_endpoint")
    print(features)
    feature_df = pd.DataFrame.from_dict([features])
    
    predictions = predictc(feature_df)
    print(predictions)
    
    result = {
        'Class' : str(predictions)
    }
    print(result)
    
    return jsonify(result)

if __name__ == "__main__":
    app.run(debug=True,host='0.0.0.0',port=9697)

Overwriting /home/ec2-user/notebooks/scripts/predict.py


In [41]:
cd scripts/

/home/ec2-user/notebooks/scripts


## Create a file to test the model

In [21]:
%%writefile '/home/ec2-user/notebooks/scripts/test_iris_model.py'
import predict
import pandas as pd
import numpy as np
import requests

"""df  = pd.DataFrame.from_dict([{
    'sepal length (cm)': 5.1,
    'sepal width (cm)' : 3.5,
    'petal length (cm)': 1.4,
    'petal width (cm)' : 0.2
}])"""
features = {
    'sepal length (cm)': 5.1,
    'sepal width (cm)' : 3.5,
    'petal length (cm)': 1.4,
    'petal width (cm)' : 0.2
}

url = 'http://localhost:9697/predict'
print(features)
response = requests.post(url,json=features)
print(response.json)

#predictions = predict.predict(df)
#print(predictions[0])

Overwriting /home/ec2-user/notebooks/scripts/test_iris_model.py


### Run below command on ec2
python predict.py

## Check the endpoint

In [22]:
%%bash
curl http://13.232.219.128:9697/predict

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   153  100   153    0     0  15504      0 --:--:-- --:--:-- --:--:-- 17000


<!doctype html>
<html lang=en>
<title>405 Method Not Allowed</title>
<h1>Method Not Allowed</h1>
<p>The method is not allowed for the requested URL.</p>


## Make predictions

In [23]:
%%bash


python test_iris_model.py

{'sepal length (cm)': 5.1, 'sepal width (cm)': 3.5, 'petal length (cm)': 1.4, 'petal width (cm)': 0.2}
<bound method Response.json of <Response [200]>>
